In [1]:
import pandas as pd
import numpy as np
import re
import math

In [2]:
df_campus = pd.read_csv('campus_loc.csv')
df_sub = pd.read_csv('suburb_detail_final.csv')

In [3]:
df_sub


,postcode,Suburb,lat,lon,Accommodation_Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person
0,3000,CBD,-37.814563,144.970267,2125.0,37323.0,5,5,Inner Melbourne,294.0
1,3002,EAST MELBOURNE,-37.816640,144.987811,48.0,1433.0,3,2,Inner Melbourne,288.0
2,3003,WEST MELBOURNE,-37.806255,144.941123,5.0,171.0,1,1,Inner Melbourne,260.0
3,3006,SOUTHBANK,-37.823258,144.965926,137.0,2751.0,4,4,Inner Melbourne,289.0
4,3008,DOCKLANDS,-37.814719,144.948039,148.0,2004.0,3,4,Inner Melbourne,308.0
5,3011,FOOTSCRAY,-37.799770,144.899587,144.0,1841.0,3,4,Outer Western Melbourne,191.0
6,3011,SEDDON,-37.808769,144.895486,36.0,603.0,1,2,Outer Western Melbourne,187.0
7,3012,WEST FOOTSCRAY,-37.797701,144.879924,41.0,998.0,2,2,Outer Western Melbourne,177.0
8,3013,YARRAVILLE,-37.816178,144.889774,72.0,1256.0,2,3,Outer Western Melbourne,187.0
9,3015,NEWPORT,-37.842477,144.883145,56.0,1285.0,3,3,Outer Western Melbourne,178.0


In [4]:
df_campus

,uni_id,University,Campus,Suburb,lat,long
0,1.0,La Trobe University,Main,Bundoora,-37.7170,145.0432
1,2.0,Monash University,Main,Clayton,-37.9153,145.1340
2,3.0,RMIT University,Main,City,-37.8081,144.9633
3,4.0,Swinburne University of Technology,Main,Hawthorn,-37.8529,144.9917
4,5.0,University of Divinity,Main,Kew,-37.8057,145.0368
5,6.0,University of Melbourne,Main,Parkville,-37.7963,144.9614
6,7.0,Victoria University,Main,Footscray,-37.7933,144.8965
7,8.0,Monash University,Caulfield,Caulfield,-37.8764,145.0454
8,9.0,Deakin University,Burwood,Burwood,-37.8500,145.1148
9,10.0,University of Melbourne,Burnley,Burnley,-37.8263,145.0173


In [5]:
df_campus = df_campus.dropna()

In [6]:
class Haversine:
    '''
    use the haversine class to calculate the distance between
    two lon/lat coordnate pairs.
    output distance available in kilometers, meters, miles, and feet.
    example usage: Haversine([lon1,lat1],[lon2,lat2]).feet
    
    '''
    def __init__(self,coord1,coord2):
        lon1,lat1=coord1
        lon2,lat2=coord2
        
        R=6378000                               # radius of Earth in meters
        phi_1=math.radians(lat1)
        phi_2=math.radians(lat2)

        delta_phi=math.radians(lat2-lat1)
        delta_lambda=math.radians(lon2-lon1)

        a=math.sin(delta_phi/2.0)**2+\
           math.cos(phi_1)*math.cos(phi_2)*\
           math.sin(delta_lambda/2.0)**2
        c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
        
        self.meters=R*c                         # output distance in meters
        self.km=self.meters/1000.0              # output distance in kilometers
        self.miles=self.meters*0.000621371      # output distance in miles
        self.feet=self.miles*5280               # output distance in feet

if __name__ == "__Haversine__":
    main()

In [7]:
cor_campus = zip(df_campus['long'],df_campus['lat'])
cor_sub = zip(df_sub['lon'],df_sub['lat'])

In [8]:
cor_campus_list = []
for i in cor_campus:
    cor_campus_list.append(i)

In [9]:
cor_sub_list = []
for i in cor_sub:
    cor_sub_list.append(i)

In [10]:
ou = []
for itemB in cor_campus_list:
    for itemA in cor_sub_list:
        ou.append(Haversine(itemB,itemA).km)

In [11]:
len(ou)

1880

In [12]:
len(df_campus)

10

In [13]:
len(df_sub)

188

In [14]:
ou

[12.615053095770337,
 12.115321934990721,
 13.394506970110319,
 13.643456773303692,
 13.727728466482477,
 15.640862257938728,
 16.53277223105325,
 16.94665329430172,
 17.440354752294486,
 19.83427401811612,
 20.158884502241257,
 25.260867515070338,
 21.823133165737026,
 21.670548708685086,
 24.366622953986592,
 35.35039904086163,
 39.742653818152974,
 12.642289456196107,
 13.218965643583342,
 12.414742904994563,
 21.840677378049655,
 24.60637768451857,
 12.190760235994182,
 11.853375854693466,
 14.404984001476471,
 14.536592230532722,
 9.239289912791689,
 8.930932258867958,
 11.307860926428916,
 11.492321623081601,
 21.06978567117314,
 11.576145304756192,
 9.89866551970783,
 9.89866551970783,
 10.244177100872694,
 7.372983138122224,
 6.481361392370003,
 13.831126064764561,
 15.91461636115481,
 13.215800044561302,
 10.909655418953875,
 10.636130484867222,
 10.216123185287588,
 9.00869833648711,
 7.239889196540782,
 5.775338825273282,
 4.4696787611555475,
 3.188989218134244,
 4.810472755

In [15]:
df_campus = df_campus.rename(index=str, columns={"Suburb": "suburb"})

In [20]:
df_sub = df_sub.rename(index=str, columns={"Suburb": "suburb"})

In [21]:
# df3 = pd.merge(df_campus, df_sub, on = 'suburb',how = 'outer')

In [22]:
# df_campus_list = []
# df_sub_list = []
#         df_campus_list.append(itemB)
#         df_sub_list.append(itemA)

In [23]:
# for uni_name in df_campus['University']:
#     for sub_name in df_sub['suburb]':

In [24]:
df_uni_list = []
df_campus_list = []
df_sub_list = []
for i in df_campus['University']:
#     for j in df_campus['Campus']:
        for k in df_sub['suburb']:
            df_sub_list.append(k)
            df_uni_list.append(i)
#             df_campus_list.append(j)


    

In [25]:
df_campus_list = []
for j in df_campus['Campus']:
    for i in df_sub['suburb']:
        df_campus_list.append(j)


In [26]:
len(df_campus_list)

1880

In [27]:
len(df_uni_list)

1880

In [28]:
len(df_sub_list)

1880

In [29]:
# df_uni_list

In [30]:
# df_sub_list

In [31]:
col_names =  ['university']
df_dis = pd.DataFrame(columns = col_names)
df_dis['university'] = df_uni_list
df_dis['campus'] = df_campus_list
df_dis['from suburb'] = df_sub_list
df_dis['distance'] = ou


In [32]:
len(df_campus_list)

1880

In [33]:
df_dis.set_index('university')

,campus,from suburb,distance
university,,,
La Trobe University,Main,CBD,12.615053
La Trobe University,Main,EAST MELBOURNE,12.115322
La Trobe University,Main,WEST MELBOURNE,13.394507
La Trobe University,Main,SOUTHBANK,13.643457
La Trobe University,Main,DOCKLANDS,13.727728
La Trobe University,Main,FOOTSCRAY,15.640862
La Trobe University,Main,SEDDON,16.532772
La Trobe University,Main,WEST FOOTSCRAY,16.946653
La Trobe University,Main,YARRAVILLE,17.440355


In [34]:
df_dis.to_csv('sub_distance.csv')